In [6]:
import pandas as pd
import numpy as np

#Create example data
np.random.seed(1)
num_houses = 20

house_names = ['house_{}'.format(i+1) for i in range(num_houses)]
house_prices = np.random.randint(100,1000,num_houses)
low_prices = np.random.randint(100,1000,num_houses)
high_prices = low_prices + np.random.randint(100,1000,num_houses)
med_prices = (low_prices+high_prices)/2

Target_Frame = pd.DataFrame({
    'Home':house_names,
    'Price':house_prices,
    'High':high_prices,
    'Low':low_prices,
    'Median':med_prices,
})



#First suggest to buy everything
Target_Frame['Suggest'] = 'Buy'

#If the price is more than the low, overwrite these rows with 'Review'
Target_Frame.loc[Target_Frame['Price'] > Target_Frame['Low'], 'Suggest'] = 'Review'

#If the price is more than the median, overwrite these rows with 'Hold'
Target_Frame.loc[Target_Frame['Price'] > Target_Frame['Median'], 'Suggest'] = 'Hold'

#If the price is more than the High, overwrite these rows with 'Sell'
Target_Frame.loc[Target_Frame['Price'] > Target_Frame['High'], 'Suggest'] = 'Sell'

Target_Frame


,Home,Price,High,Low,Median,Suggest
0,house_1,137,1006,590,798.0,Buy
1,house_2,335,1077,768,922.5,Buy
2,house_3,172,862,498,680.0,Buy
3,house_4,867,1490,662,1076.0,Review
4,house_5,815,1433,680,1056.5,Review
5,house_6,745,1042,315,678.5,Hold
6,house_7,947,1384,853,1118.5,Review
7,house_8,244,1336,603,969.5,Buy
8,house_9,229,1134,578,856.0,Buy
9,house_10,683,1606,964,1285.0,Buy


,Home,Price,High,Low,Median,Suggest
0,house_1,882,1101,711,906.0,Review
1,house_2,526,755,210,482.5,Hold
2,house_3,389,1288,947,1117.5,Buy
3,house_4,675,943,281,612.0,Hold
4,house_5,542,1003,782,892.5,Buy
5,house_6,614,1518,945,1231.5,Buy
6,house_7,386,1872,927,1399.5,Buy
7,house_8,858,1388,420,904.0,Review
8,house_9,325,1300,325,812.5,Buy
9,house_10,794,1310,680,995.0,Review


In [39]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy as scp

# Getting the protein sequences to run DeepSig with

Downloaded the translated amino-acid sequence fasta from https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_mouse/release_M28/gencode.vM28.pc_translations.fa.gz on 3/8/2022

In [114]:
with open('../preprocessing/MOP_preprocessing/mop_genes.txt') as mop_gene_f:
    mop_genes = set([g.strip() for g in mop_gene_f.readlines()])
    
with open('../preprocessing/Vizgen_preprocessing/vizgen_genes.txt') as viz_gene_f:
    viz_genes = set([g.strip() for g in viz_gene_f.readlines()])

with open('../preprocessing/SeqFishplus_preprocessing/seq_fish_plus_genes.txt') as seq_gene_f:
    seq_genes = set([g.strip() for g in seq_gene_f.readlines()])

    
genes = mop_genes.union(viz_genes).union(seq_genes)
print(len(genes))

10375


In [128]:
import pysam

path = '/oak/stanford/groups/horence/rob/isoform_localizations/deepsig/gencode.vM28.pc_translations.fa'

seqs = {}

with pysam.FastaFile(path) as fa:
    for ref in fa.references:
        gene = ref.split('|')[-2]
        if gene not in genes:
            continue
        
        seq = fa.fetch(reference=ref)
        
        #use the longest isoform for the signal peptide prediction        
        if (gene not in seqs) or (len(seq) > len(seqs[gene])):
            seqs[gene] = seq
            
        
len(seqs)

9974

In [129]:
#Nearly all the MOp genes were found
print(len(mop_genes))
len(mop_genes.intersection(seqs.keys()))

252


238

In [130]:
#Most of the Viz genes were found
print(len(viz_genes))
len(viz_genes.intersection(seqs.keys()))

649


478

In [132]:
#Most of the Seq genes were found
print(len(seq_genes))
len(seq_genes.intersection(seqs.keys()))

10000


9772

In [133]:
with open('/oak/stanford/groups/horence/rob/isoform_localizations/deepsig/sub_prots.fa','w') as fa:
    for gene,seq in seqs.items():
        fa.write('>{}\n{}\n'.format(gene,seq))

# Notes on running Deepsig

Working in this directory: /oak/stanford/groups/horence/rob/isoform_localizations/deepsig

Have a virtualenv with deepsig installed: source .venv/bin/activate

(need to ml python/3.9.0 too)

then deepsig -h to get the help menu

I ran deepsig with: deepsig -f sub_prots.fa -k euk -o results.txt

Then copied the outputs to SRRS/inputs/deepsig_results.csv